In [ ]:
import cv2
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
import tensorflow.keras.layers as layers
from data_generator import DataGenerator
import pandas as pd
from __future__ import absolute_import, division, print_function
import os
import sys
print(os.getcwd())
os.environ["CUDA_VISIBLE_DEVICES"]= sys.argv[1]
import tensorflow as tf
#tf.compat.v1.enable_eager_execution()
import numpy as np
#from keras.datasets import cifar10
import keras.callbacks as callbacks
#import keras.utils.np_utils as kutils
from tensorflow.keras import utils
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import plot_model
#from wide_resnet import WRNModel
from keras import backend as K
from keras.callbacks import ModelCheckpoint, CSVLogger
from keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import h5py
#from keras.utils import plot_model
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
tf.compat.v1.disable_eager_execution()
# parts of the code were adapted from https://github.com/jemiar/surgery-gesture-recog
# parts of the code were adapted from https://github.com/yashkant/padam-tensorflow

In [ ]:
sys.path.append(os.path.dirname(os.getcwd()))
sys.path.append(os.getcwd())

In [ ]:
base_directory = '/Users/boabangfrancis/'

In [ ]:
import zipfile
with zipfile.ZipFile("/Users/boabangfrancis/Suturing.zip","r") as zip_ref:
    zip_ref.extractall("/Users/boabangfrancis/Suturing")

In [ ]:
import os
#h=[]
#for i in os.listdir('/Users/boabangfrancis/transcription/transcription/'):
#    if i.startswith('S'):
 #       h.append(i)
#transcriptions=sorted(h[16:17])
transcriptions=['Suturing_B001.csv', 'Suturing_C001.csv','Suturing_D001.csv','Suturing_E001.csv', 'Suturing_G001.csv','Suturing_H001.csv','Suturing_I001.csv']

In [ ]:
def save_data(fromarray=transcriptions, height=240, width=320, folder='data_action/', idnumber=1):
    # array to store ids of normal blocks
    normals = []
    # array to store ids of transit blocks
    transits = []
    # dictionary to store target y value (class) of each block
    labels = {}
    os.chdir(base_directory + 'Suturing/Suturing/video/')
    # for each element in fromarray (store video file names)
    for arr in fromarray:
        # use CV2 to capture the video file
        cap = cv2.VideoCapture(arr[:][:-4] + '_capture1.avi')
        i = 1
        # Initialize numpy array to store frames of Red, Green, Blue channels of video
        red_frames = np.empty((0, height, width))
        green_frames = np.empty((0, height, width))
        blue_frames = np.empty((0, height, width))
        # while reading the capture, only store 1 frame for every 3 frames
        while cap.isOpened():
            ret, frame = cap.read()
            if ret == False:
                break
            if i%3 == 1:
                # Resize the frame to reduce the computation during training
                frame = cv2.resize(frame, (width, height), interpolation=cv2.INTER_AREA)
                # Cast the frame as a numpy array
                f = np.asarray(frame)
                # Update the color of frame from BGR to RGB
                f = cv2.cvtColor(f, cv2.COLOR_BGR2RGB)
                # Apprend frame to its appropriate channel
                red_frames = np.append(red_frames, np.expand_dims(f[:,:,0], axis=0), axis=0)
                green_frames = np.append(green_frames, np.expand_dims(f[:,:,1], axis=0), axis=0)
                blue_frames = np.append(blue_frames, np.expand_dims(f[:,:,2], axis=0), axis=0)
            i += 1
        # Release the capture when finishing reading
        cap.release()
        # Normalize the value of each element to range [0, 1]
        red_frames = red_frames / 255.0
        green_frames = green_frames / 255.0
        blue_frames = blue_frames / 255.0
        # For each transciption (transcribe where each gesture starts and ends)
        df= pd.read_csv(base_directory + 'transcription/transcription/' + str(arr[:-4]) + '.csv') 
        arr_gesture=np.array(df)
        for k, t in enumerate(arr_gesture[10:-4]):
            # Save the normal block
            # Calculate the left most frame of 1 gesture
            left = (int((t[0])) + 1) // 3
            # Calculate the right most frame of 1 gesture
            right = (int((t[1])) - 1) // 3
            # Calculate the number of normal blocks in a gesture
            num_blocks = (right - left + 1) // 10
            for index in range(num_blocks):
                block = np.expand_dims(red_frames[left+index*10:left+(index+1)*10,:,:], axis=3)
                block = np.append(block, np.expand_dims(green_frames[left+index*10:left+(index+1)*10,:,:], axis=3), axis=3)
                block = np.append(block, np.expand_dims(blue_frames[left+index*10:left+(index+1)*10,:,:], axis=3), axis=3)
            # Store normal block
                npy_name = 'id_' + str(idnumber)
                temp_obj = {'id': npy_name, 'file': arr[:], 'label': 0}
                normals.append(npy_name)
                labels[npy_name] = 0
                np.save(base_directory + folder + npy_name + '.npy', block)
                idnumber += 1

            # Save transit blocks
            if k < (len(arr_gesture[10:-4]) - 1):
                # Each transit block has the last 5 frames of 1 gesture and the 1st 5 frames of the next gesture
                # Calculate the left most frame of a transit block
                ind = (int(t[1]) - 1) // 3 - 4
                block = np.expand_dims(red_frames[ind:ind+10,:,:], axis=3)
                block = np.append(block, np.expand_dims(green_frames[ind:ind+10,:,:], axis=3), axis=3)
                block = np.append(block, np.expand_dims(blue_frames[ind:ind+10,:,:], axis=3), axis=3)
                # Store transit block
                #npy_name = 'id_' + str(idnumber)
                temp_obj = {'id': npy_name, 'file': arr[:], 'label': 1}
                normals.append(npy_name)
                labels[npy_name] = 1
                np.save(base_directory + folder + npy_name + '.npy', block)
                idnumber += 1

    return normals, labels

In [ ]:
def create_model(height=240, width=320):
    # shape of input: 1 block has 10 frames x height x width x 3 channels (RGB)
    input = tf.keras.Input((10, height, width, 3))

    # 1st Conv3D block includes Conv3D with 8 filters, MaxPool3D and BatchNormalization
    x = layers.Conv3D(filters=8, kernel_size=(3,3,3), activation='relu')(input)
    x = layers.MaxPool3D(pool_size=(2,2,2))(x)
    x = layers.BatchNormalization()(x)

    # 2nd Conv3D block includes Conv3D with 16 filters, MaxPool3D and BatchNormalization
    x = layers.Conv3D(filters=16, kernel_size=(3,3,3), activation='relu')(x)
    x = layers.MaxPool3D(pool_size=(2,2,2))(x)
    x = layers.BatchNormalization()(x)

    # 3rd Conv3D block includes Conv3D with 32 filters, MaxPool3D and BatchNormalization
    x = layers.Conv3D(filters=32, kernel_size=(3,3,3), activation='relu')(input)
    x = layers.MaxPool3D(pool_size=(1,2,2))(x)
    x = layers.BatchNormalization()(x)

    # Fully-connected block includes GlobalAveragePooling3D, Fully-Connected layer with 512 units and DropOut for Regularization
    x = layers.GlobalAveragePooling3D()(x)
    x = layers.Dense(units=512, activation='relu')(x)
    x = layers.Dropout(0.7)(x)

    # output shape (1,) produces value between [0, 1]
    output = layers.Dense(units=1, activation='sigmoid')(x)

    model = tf.keras.Model(input, output, name='3DCNN')
    return model

Create model

In [ ]:
# Create data generator for training and validation

#normals, labels =save_data(fromarray=transcriptions, height=240, width=320, folder='data_action/', idnumber=1)

In [ ]:
optim_params = {
    
    
    'proposed_ASGD': {
        'weight_decay': 0.0001,
        'lr_min': 0.00001,
        'lr_max':0.0001,
        #'lr': 0, # dummy value
        'b1': 0.9,
        'b2': 0.99,
        'color': 'red',
        'linestyle':'-'
    },
    
    
    
    'padam': {
        'weight_decay': 0.0001,
        'lr_max': 0.001,
        'p': 0.2,
        'b1': 0.9,
        'b2': 0.999,
        'color': 'darkred',
        'linestyle':'-'
    },
    'adam': {
        'weight_decay': 0.0001,
        'lr_min': 0.00001,
        'b1': 0.9,
        'b2': 0.99,
        'color': 'orange',
        'linestyle':'--'
    },
    'adamw': {
        'weight_decay': 0.025,
        'lr_min': 0.00001,
        'b1': 0.9,
        'b2': 0.99,
        'color': 'magenta',
        'linestyle':'--'
    },
    'amsgrad': {
        'weight_decay': 0.0001,
        'lr_min': 0.00001,
        'b1': 0.9,
        'b2': 0.99,
        'color' : 'darkgreen',
        'linestyle':'-.'
    },
    'sgd': {
        'weight_decay': 0.0005,
        'lr_max': 0.0001,
        'm': 0.9,
        'color': 'blue',
        'linestyle':'-'
    }
}

In [ ]:
from sklearn.utils import resample
#ham_messages=[{'id_1','id_1','id_1','id_1','id_3','id_1','id_3','id_2','id_1','id_1','id_4','id_1','id_1'}]
ham_downsample = resample([labels],
             replace=True,
             n_samples=len([labels]),
             random_state=42)

In [ ]:
label={'id_1': 0,
  'id_2': 0,
  'id_3': 0,
  'id_4': 0,
  'id_5': 0,
  'id_6': 0,
  'id_7': 0,
  'id_8': 0,
  'id_9': 0,
  'id_10': 0,
  'id_11': 0,
  'id_12': 0,
  'id_13': 0,
  'id_14': 0,
  'id_15': 0,
  'id_16': 1,
  'id_18': 0,
  'id_19': 0,
  'id_20': 0,
  'id_21': 0,
  'id_22': 0,
  'id_23': 0,
  'id_24': 1,
  'id_26': 0,
  'id_27': 0,
  'id_28': 0,
  'id_29': 0,
  'id_30': 0,
  'id_31': 1,
  'id_33': 0,
  'id_34': 1,
  'id_36': 0,
  'id_37': 0,
  'id_38': 0,
  'id_39': 0,
  'id_40': 0,
  'id_41': 0,
  'id_42': 1,
  'id_44': 0,
  'id_45': 0,
  'id_46': 0,
  'id_47': 0,
  'id_48': 0,
  'id_49': 1,
  'id_51': 0,
  'id_52': 0,
  'id_53': 0,
  'id_54': 0,
  'id_55': 1,
  'id_57': 0,
  'id_58': 0,
  'id_59': 0,
  'id_60': 0,
  'id_61': 0,
  'id_62': 0,
  'id_63': 0,
  'id_64': 0,
  'id_65': 0,
  'id_66': 0,
  'id_67': 1,
  'id_69': 0,
  'id_70': 0,
  'id_71': 0,
  'id_72': 0,
  'id_73': 0,
  'id_74': 0,
  'id_75': 0,
  'id_76': 0,
  'id_77': 0,
  'id_78': 0,
  'id_79': 1,
  'id_81': 0,
  'id_82': 0,
  'id_83': 0,
  'id_84': 0,
  'id_85': 0,
  'id_86': 0,
  'id_87': 0,
  'id_88': 0,
  'id_89': 0,
  'id_90': 1,
  'id_92': 1,
  'id_94': 0,
  'id_95': 0,
  'id_96': 0,
  'id_97': 0,
  'id_98': 0,
  'id_99': 0,
  'id_100': 0,
  'id_101': 0,
  'id_102': 0,
  'id_103': 0,
  'id_104': 1,
  'id_106': 0,
  'id_107': 0,
  'id_108': 0,
  'id_109': 1,
  'id_111': 0,
  'id_112': 0,
  'id_113': 0,
  'id_114': 0,
  'id_115': 0,
  'id_116': 0,
  'id_117': 0,
  'id_118': 1,
  'id_120': 0,
  'id_121': 0,
  'id_122': 1,
  'id_124': 0,
  'id_125': 0,
  'id_126': 0,
  'id_127': 0,
  'id_128': 0,
  'id_129': 0,
  'id_130': 1,
  'id_132': 0,
  'id_133': 0,
  'id_134': 0,
  'id_135': 0,
  'id_136': 1,
  'id_138': 0,
  'id_139': 0,
  'id_140': 0,
  'id_141': 0,
  'id_142': 1,
  'id_144': 0,
  'id_145': 1,
  'id_147': 0,
  'id_148': 0,
  'id_149': 1,
  'id_151': 0,
  'id_152': 0,
  'id_153': 0,
  'id_154': 0,
  'id_155': 0,
  'id_156': 1,
  'id_158': 1,
  'id_160': 1,
  'id_162': 0,
  'id_163': 0,
  'id_164': 0,
  'id_165': 0,
  'id_166': 0,
  'id_167': 0,
  'id_168': 0,
  'id_169': 1,
  'id_171': 0,
  'id_172': 0,
  'id_173': 0,
  'id_174': 0,
  'id_175': 0,
  'id_176': 0,
  'id_177': 0,
  'id_178': 0,
  'id_179': 0,
  'id_180': 1,
  'id_182': 0,
  'id_183': 0,
  'id_184': 0,
  'id_185': 0,
  'id_186': 0,
  'id_187': 0,
  'id_188': 0,
  'id_189': 1,
  'id_191': 0,
  'id_192': 0,
  'id_193': 0,
  'id_194': 0,
  'id_195': 0,
  'id_196': 1,
  'id_198': 0,
  'id_199': 0,
  'id_200': 0,
  'id_201': 1,
  'id_203': 0,
  'id_204': 0,
  'id_205': 0,
  'id_206': 1,
  'id_208': 0,
  'id_209': 0,
  'id_210': 0,
  'id_211': 0,
  'id_212': 0,
  'id_213': 0,
  'id_214': 0,
  'id_215': 0,
  'id_216': 0,
  'id_217': 0,
  'id_218': 0,
  'id_219': 0,
  'id_220': 1,
  'id_222': 1,
  'id_224': 0,
  'id_225': 1,
  'id_227': 0,
  'id_228': 0,
  'id_229': 0,
  'id_230': 0,
  'id_231': 0,
  'id_232': 0,
  'id_233': 0,
  'id_234': 0,
  'id_235': 0,
  'id_236': 0,
  'id_237': 0,
  'id_238': 0,
  'id_239': 0,
  'id_240': 0,
  'id_241': 1,
  'id_243': 0,
  'id_244': 0,
  'id_245': 0,
  'id_246': 0,
  'id_247': 0,
  'id_248': 1,
  'id_250': 0,
  'id_251': 0,
  'id_252': 0,
  'id_253': 1,
  'id_255': 0,
  'id_256': 0,
  'id_257': 0,
  'id_258': 1,
  'id_260': 0,
  'id_261': 0,
  'id_262': 0,
  'id_263': 0,
  'id_264': 1,
  'id_266': 0,
  'id_267': 0,
  'id_268': 0,
  'id_269': 1,
  'id_271': 0,
  'id_272': 0,
  'id_273': 0,
  'id_274': 0,
  'id_275': 1,
  'id_277': 0,
  'id_278': 0,
  'id_279': 0,
  'id_280': 0,
  'id_281': 0,
  'id_282': 0,
  'id_283': 0,
  'id_284': 0,
  'id_285': 0,
  'id_286': 0,
  'id_287': 0,
  'id_288': 0,
  'id_289': 0,
  'id_290': 0,
  'id_291': 0,
  'id_292': 0,
  'id_293': 0,
  'id_294': 0,
  'id_295': 0,
  'id_296': 0,
  'id_297': 1,
  'id_299': 0,
  'id_300': 0,
  'id_301': 0,
  'id_302': 0,
  'id_303': 0,
  'id_304': 0,
  'id_305': 1,
  'id_307': 0,
  'id_308': 0,
  'id_309': 0,
  'id_310': 0,
  'id_311': 0,
  'id_312': 0,
  'id_313': 0,
  'id_314': 0,
  'id_315': 0,
  'id_316': 0,
  'id_317': 0,
  'id_318': 0,
  'id_319': 1,
  'id_321': 0,
  'id_322': 0,
  'id_323': 0,
  'id_324': 0,
  'id_325': 1,
  'id_327': 0,
  'id_328': 0,
  'id_329': 1,
  'id_331': 0,
  'id_332': 0,
  'id_333': 0,
  'id_334': 0,
  'id_335': 0,
  'id_336': 0,
  'id_337': 0,
  'id_338': 0,
  'id_339': 0,
  'id_340': 0,
  'id_341': 0,
  'id_342': 0,
  'id_343': 0,
  'id_344': 0,
  'id_345': 0,
  'id_346': 0,
  'id_347': 0,
  'id_348': 0,
  'id_349': 0,
  'id_350': 0,
  'id_351': 0,
  'id_352': 0,
  'id_353': 0,
  'id_354': 0,
  'id_355': 0,
  'id_356': 0,
  'id_357': 0,
  'id_358': 0,
  'id_359': 0,
  'id_360': 0,
  'id_361': 0,
  'id_362': 0,
  'id_363': 0,
  'id_364': 0,
  'id_365': 0,
  'id_366': 1,
  'id_369': 0,
  'id_370': 0,
  'id_371': 0,
  'id_372': 0,
  'id_373': 0,
  'id_374': 0,
  'id_375': 0,
  'id_376': 0,
  'id_377': 0,
  'id_378': 0,
  'id_379': 0,
  'id_380': 0,
  'id_381': 0,
  'id_382': 0,
  'id_383': 0,
  'id_384': 0,
  'id_385': 0,
  'id_386': 0,
  'id_387': 0,
  'id_388': 1,
  'id_390': 1,
  'id_392': 0,
  'id_393': 0,
  'id_394': 0,
  'id_395': 0,
  'id_396': 1,
  'id_398': 1,
  'id_400': 0,
  'id_401': 1,
  'id_403': 0,
  'id_404': 1,
  'id_406': 0,
  'id_407': 0,
  'id_408': 0,
  'id_409': 0,
  'id_410': 0,
  'id_411': 0,
  'id_412': 0,
  'id_413': 1,
  'id_415': 0,
  'id_416': 0,
  'id_417': 0,
  'id_418': 0,
  'id_419': 0,
  'id_420': 0,
  'id_421': 1,
  'id_423': 0,
  'id_424': 0,
  'id_425': 0,
  'id_426': 0,
  'id_427': 0,
  'id_428': 0,
  'id_429': 0,
  'id_430': 0,
  'id_431': 0,
  'id_432': 0,
  'id_433': 1,
  'id_435': 0,
  'id_436': 0,
  'id_437': 0,
  'id_438': 0,
  'id_439': 0,
  'id_440': 1,
  'id_442': 0,
  'id_443': 0,
  'id_444': 0,
  'id_445': 0,
  'id_446': 0,
  'id_447': 0,
  'id_448': 0,
  'id_449': 0,
  'id_450': 1,
  'id_452': 0,
  'id_453': 0,
  'id_454': 0,
  'id_455': 0,
  'id_456': 0,
  'id_457': 0,
  'id_458': 0,
  'id_459': 0,
  'id_460': 0,
  'id_461': 0,
  'id_462': 0,
  'id_463': 1,
  'id_465': 0,
  'id_466': 0,
  'id_467': 0,
  'id_468': 1,
  'id_470': 0,
  'id_471': 0,
  'id_472': 0,
  'id_473': 0,
  'id_474': 0,
  'id_475': 0,
  'id_476': 0,
  'id_477': 1,
  'id_479': 0,
  'id_480': 0,
  'id_481': 0,
  'id_482': 0,
  'id_483': 0,
  'id_484': 0,
  'id_485': 0,
  'id_486': 0,
  'id_487': 0,
  'id_488': 0,
  'id_489': 0,
  'id_490': 0,
  'id_491': 0,
  'id_492': 0,
  'id_493': 0}

In [ ]:
def save_model(filepath, model):
    file = h5py.File(filepath,'w')
    weight = model.get_weights()
    for i in range(len(weight)):
        file.create_dataset('weight'+str(i),data=weight[i])
    file.close()

In [ ]:
def load_model(filepath, model):
    file=h5py.File(filepath,'r')
    weight = []
    for i in range(len(file.keys())):
        weight.append(file['weight'+str(i)][:])
    model.set_weights(weight)
    return model 

In [ ]:
#dimension=np.load('/Users/boabangfrancis/data_action/' + train)
#dim_shape=(dimension.shape[0],dimension.shape[1], dimension.shape[2])
labelssorted=dict(sorted(labels.items()))
params = {'dim': (10, 240, 320),
          'batch_size': 10,
          'n_classes': 2,
          'n_channels': 3,
          'folder': 'data_action/',
          'shuffle': True}
  
#y = tf.keras.layers.UpSampling3D(size=10)(normals)   
    
train_generator = DataGenerator(normals[101:], label, **params)
val_generator = DataGenerator(normals[1:100], labels, **params)

In [ ]:
from padam import Padam
from amsgrad import AMSGrad
from proposed_ASGD import Proposed_ASGD
#learning_rate=0.00001
#learning_rate_min=0.00001
#learning_rate_max=0.001
optim_array=[ 'padam', 'adam', 'proposed_ASGD',  'amsgrad', 'sgd']
history={}
for optimizer in optim_array:
    op=optim_params[optimizer]
    model = create_model(240, 320) 
    logfile= 'log_' + optimizer +'_' + '.csv'
    #op['lr']=op['lr']
   # learning_rate_min=op['lr_min']
    
    if optimizer == 'proposed_ASGD':
        learning_rate_max=op['lr_max']
        learning_rate_min=op['lr_min']
        optim = Proposed_ASGD(learning_rate_min=learning_rate_min,learning_rate_max=learning_rate_max, beta1=op['b1'], beta2=op['b2'])
        
    elif optimizer == 'padam':
        learning_rate_max=op['lr_max']
        optim = Padam(learning_rate=learning_rate_max, p=op['p'], beta1=op['b1'], beta2=op['b2'])
    elif optimizer == 'adam':
        learning_rate_min=op['lr_min']
        optim = tf.train.AdamOptimizer(learning_rate=learning_rate_min, beta1=op['b1'], beta2=op['b2'])
    elif optimizer == 'adamw':
        learning_rate_min=op['lr_min']
            # adamw = tf.contrib.opt.extend_with_decoupled_weight_decay(tf.train.AdamOptimizer)
        optim = tf.contrib.opt.AdamWOptimizer(weight_decay=op['weight_decay'], learning_rate=learning_rate_min,  beta1=op['b1'], beta2=op['b2'])
    elif optimizer == 'amsgrad':
        learning_rate_min=op['lr_min']
        optim = AMSGrad(learning_rate=learning_rate_min, beta1=op['b1'], beta2=op['b2'])
    elif optimizer == 'sgd':
        learning_rate_max=op['lr_max']
        optim = tf.train.MomentumOptimizer(learning_rate=learning_rate_max, momentum=op['m'])
       
    model.compile(optimizer=optim, loss='binary_crossentropy', metrics=['accuracy', 'top_k_categorical_accuracy'])
    csv_logger = CSVLogger(logfile, append=True, separator=';')
        #model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(), optimizer=keras.optimizers.SGD(learning_rate=learning_rate), metrics=metrics)
    model.fit_generator(generator=train_generator, validation_data=val_generator, epochs=100, shuffle=False, verbose=1, callbacks = [csv_logger])
    scores = model.evaluate_generator(val_generator, verbose=1)
    print("Final test loss and accuracy:", scores)
    #save_model(save_model_filepath, model)

In [ ]:
normals.shape

In [ ]:
pwd

In [ ]:
cd ..

In [ ]:
pwd